In [ ]:
import urllib
import urllib.request

def get_page(url):
    return urllib.request.urlopen(url).read().decode("iso-8859-1")

def is_int(txt):
    try:
        int(txt)
        return True
    except:
        return False

In [ ]:
last_page = [i.split('"')[0] for i in get_page("https://en.grandgames.net/kakuro/"
                                              ).replace("'",'"').split('en.grandgames.net/kakuro/')]
last_page = [int(i) for i in last_page if is_int(i)]
last_page.sort()
last_page = last_page[-1]

nums = set()
for i in range(1, last_page+1):
    url = 'https://en.grandgames.net/kakuro/%d'%i
    print('\n'+url, end=": ")
    html = get_page(url)
    for i in html.split('/kakuro/id')[1:]:
        print(' ',i.split('"')[0].split("'")[0], end='')
        nums.add(i.split('"')[0].split("'")[0])
        
nums = [int(i) for i in nums]
nums.sort()
with open('kakuronums.txt','w') as f:
    f.write('\n'.join([str(i) for i in nums]))

In [ ]:
def extract_kakuro(url, outfname):
    html = get_page(url)
    str_in = html.split('sudoku_n_tbl')[1].split('script')[0]
    items = [i.replace('"','').replace("'",'').split('<')[0].split('>')[:2] for i in str_in.split('id=')[1:]][1:]
    import re
    import numpy
    from pprint import pprint

    rows,cols = (0,0)
    for k,v in items:
        i, j = [int(i) for i in re.findall(r'\d+', k)]
        rows = max(rows, i+1)
        cols = max(cols, j+1)

        #print((i,j),k,repr(v))

    cells = ([[[None,None] for i in range(cols)] for i in range(rows)])
    for k,v in items:
        i, j = [int(i) for i in re.findall(r'\d+', k)]
        if 'h' in k:
            cells[i][j][1] = v
        if 'v' in k:
            cells[i][j][0] = v
        if 'h' not in k and 'v' not in k:
            cells[i][j][0] = ''
            cells[i][j][1] = ''

    for i in range(rows):
        for j in range(cols):
            if cells[i][j] == [None,None]:
                cells[i][j] = '.....'
            else:
                if not cells[i][j][0]:
                    cells[i][j][0] = ' _'

                if not cells[i][j][1]:
                    cells[i][j][1] = '_ '

                cells[i][j] = '%2s\\%2s'%(cells[i][j][0], (cells[i][j][1]+' ')[:2])

            if cells[i][j] == ' _\\_ ':
                cells[i][j] = '     '


    with open(outfname, 'w') as f:
        f.write('#source '+url+'\n\n')
        for i, line in enumerate(cells):
            if i!=0: 
                for j in range((cols*5+cols-1)):
                    if (j+1)%6==0:
                        f.write('+')
                    else:
                        f.write('-')
                f.write('\n')

            for i in line:
                if len(i) != 5:
                    print(i)
            f.write('|'.join(line)+'\n')
            
            
import time
nums = [int(i) for i in open('kakuronums.txt').read().split()]
while(True):
    try:
        for i in nums:
            fname = '../kakuro-collection/kakuro-grandgames.net-%d.txt'%i
            try:
                fin = open(fname).read()
            except:
                pass
            else:
                if len(fin) >= 100:
                    continue
                
            print(fname)
            url = 'https://en.grandgames.net/kakuro/id%d'%i
            extract_kakuro(url, fname)
        break
             
    except:
        print('timeout...')
        time.sleep(5)